In [ ]:

# Commented out IPython magic to ensure Python compatibility.
import os
import sys
import numpy
# from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from skimage import io
from skimage.transform import resize
from numpy import asarray
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd 
# %matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf


batch_size = 4
n_classes = 5
input_shape = 512, 512, 3

trainLabels = pd.read_csv("/Volumes/surya/aptos2019-blindness-detection/train.csv")
trainLabels.head()

img_rows, img_cols = 512, 512
from skimage import color
path = 'Volumes/surya/aptos2019-blindness-detection/train_images/'
immatrix = []
imlabel = []
dirs = [l for l in os.listdir(path) if l != '.DS_Store']
total = 0
for item in dirs:
    base = os.path.basename(path + item)
    #print(base)
    fileName = os.path.splitext(base)[0]
    #print(fileName)
    total +=1
    #print(dirs)
    try:
      ls = trainLabels.loc[trainLabels.id_code==fileName, 'diagnosis']
      lb = trainLabels.values[ls]
      ld = imlabel.append(ls)
      print(imlabel)
      print(fileName," ",total)
      import cv2
      import glob
      import imageio
      from skimage.color import rgb2gray
      #imgs = glob.glob('/content/train_images/*.png')
      # for i in imgs:
      #   img = cv2.imread(i)
      #   resize_img = cv2.resize(img, (512, 512))
      #   cv2.imwrite(i, resize_img)
      img = cv2.imread(path +item)
      resize_img = cv2.resize(img, (512,512))
      img_gray = cv2.cvtColor(np.asarray(img), cv2.COLOR_BGR2GRAY)
      img_gray.flatten()
      cv2.imwrite("gray"+'/Volumes/surya/aptos2019-blindness-detection/train_images/*.png',img_gray)
      pixels = img_gray.astype(np.float32)
      immatrix.append(pixels)
      #plt.imshow(img)
    except (IOError,ValueError) as e:
      print('could not read the',fileName ,':',e,'hence skipping it.')
immatrix = np.asarray(immatrix)
imlabel = np.asarray(imlabel)

from sklearn.utils import shuffle

data,Label = shuffle(immatrix,imlabel, random_state=4)
train_data = [data,Label]
type(train_data)

(X, y) = (train_data[0],train_data[1])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols,1)
x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols,1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols,1)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_val /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

from keras.utils import np_utils

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, n_classes)
y_val = np_utils.to_categorical(y_val, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

#data augmentation
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(   
        rotation_range=10, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        shear_range=0.15,  
        zoom_range=0.1,  
        fill_mode='nearest', 
        horizontal_flip=True)
datagen.fit(x_train)

train_gen = datagen.flow(x_train, y_train, batch_size=batch_size)

#EfficientNetB0 Model

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import json
import math
import string
import collections
import numpy as np
from keras import backend as K
K.set_image_data_format('channels_last')
from six.moves import xrange
from keras_applications.imagenet_utils import _obtain_input_shape
from keras_applications.imagenet_utils import decode_predictions
from keras_applications.imagenet_utils import preprocess_input as _preprocess_input
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Reshape, DepthwiseConv2D, Lambda, multiply, add, GlobalAveragePooling2D
from keras.layers import Concatenate, SpatialDropout2D
from keras.optimizers import Adam, SGD
from keras import regularizers
#from . import get_submodules_from_kwargs
import os
import math
from typing import List
from keras import backend as K
from keras import layers
from keras.models import Model
from keras.layers import Input
from keras.utils import get_file, get_source_inputs
_KERAS_BACKEND = None
_KERAS_LAYERS = None
_KERAS_MODELS = None
_KERAS_UTILS = None
from keras_applications.imagenet_utils import _obtain_input_shape
from keras_applications.imagenet_utils import preprocess_input as _preprocess

def get_submodules_from_kwargs(kwargs):
    backend = kwargs.get('backend', _KERAS_BACKEND)
    layers = kwargs.get('layers', _KERAS_LAYERS)
    models = kwargs.get('models', _KERAS_MODELS)
    utils = kwargs.get('utils', _KERAS_UTILS)
    for key in kwargs.keys():
        if key not in ['backend', 'layers', 'models', 'utils']:
            raise TypeError('Invalid keyword argument: %s', key)
    return backend, layers, models, utils

BlockArgs = collections.namedtuple('BlockArgs', [
    'kernel_size', 'num_repeat', 'input_filters', 'output_filters',
    'expand_ratio', 'id_skip', 'strides', 'se_ratio'
])
BlockArgs.__new__.__defaults__ = (None,) * len(BlockArgs._fields)
DEFAULT_BLOCKS_ARGS = [
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16,
              expand_ratio=1, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, output_filters=40,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=3, input_filters=40, output_filters=80,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=3, input_filters=80, output_filters=112,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=4, input_filters=112, output_filters=192,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=192, output_filters=320,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25)
]

'''
BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16,
              expand_ratio=1, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, output_filters=40,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=3, input_filters=40, output_filters=80,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=3, input_filters=80, output_filters=112,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=4, input_filters=112, output_filters=192,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=192, output_filters=320,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25)
'''

CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        # EfficientNet actually uses an untruncated normal distribution for
        # initializing conv layers, but keras.initializers.VarianceScaling use
        # a truncated distribution.
        # We decided against a custom initializer for better serializability.
        'distribution': 'normal'
    }
}

DENSE_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 1. / 3.,
        'mode': 'fan_out',
        'distribution': 'uniform'
    }
}

def get_swish(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)
    def swish(x):
        
        return x * Activation('sigmoid')(x)
    return  swish

def round_filters(filters, width_coefficient, depth_divisor):
    """Round number of filters based on width multiplier."""

    filters *= width_coefficient
    new_filters = int(filters + depth_divisor / 2) // depth_divisor * depth_divisor
    new_filters = max(depth_divisor, new_filters)
    # Make sure that round down does not go down by more than 10%.
    if new_filters < 0.9 * filters:
        new_filters += depth_divisor
    return int(new_filters)

def round_repeats(repeats, depth_coefficient):
    """Round number of repeats based on depth multiplier."""

    return int(math.ceil(depth_coefficient * repeats))

def mb_conv_block(inputs, block_args, activation, drop_rate=None, prefix='', ):
    """Mobile Inverted Residual Bottleneck."""

    has_se = (block_args.se_ratio is not None) and (0 < block_args.se_ratio <= 1)
    bn_axis = 3 if K.set_image_data_format == 'channels_last' else 1


    # Expansion phase
    filters = block_args.input_filters * block_args.expand_ratio
    if block_args.expand_ratio != 1:
        x = Conv2D(filters, 1,
                          padding='same',
                          use_bias=False,
                          kernel_initializer=CONV_KERNEL_INITIALIZER,
                          name=prefix + 'expand_conv')(inputs)
        x = BatchNormalization(axis=bn_axis, name=prefix + 'expand_bn')(x)
        x = Activation(activation, name=prefix + 'expand_activation')(x)
    else:
        x = inputs

    # Depthwise Convolution
    x = DepthwiseConv2D(block_args.kernel_size,
                               strides=block_args.strides,
                               padding='same',
                               use_bias=False,
                               depthwise_initializer=CONV_KERNEL_INITIALIZER,
                               name=prefix + 'dwconv')(x)
    x = BatchNormalization(axis=bn_axis, name=prefix + 'bn')(x)
    x = Activation(activation, name=prefix + 'activation')(x)

    # Squeeze and Excitation phase
    if has_se:
        num_reduced_filters = max(1, int(
            block_args.input_filters * block_args.se_ratio
        ))
        se_tensor = GlobalAveragePooling2D(name=prefix + 'se_squeeze')(x)

        target_shape = (1, 1, filters)
        se_tensor = Reshape(target_shape, name=prefix + 'se_reshape')(se_tensor)
        se_tensor = Conv2D(num_reduced_filters, 1,
                                  activation=activation,
                                  padding='same',
                                  use_bias=True,
                                  kernel_initializer=CONV_KERNEL_INITIALIZER,
                                  name=prefix + 'se_reduce')(se_tensor)
        se_tensor = Conv2D(filters, 1,
                                  activation='sigmoid',
                                  padding='same',
                                  use_bias=True,
                                  kernel_initializer=CONV_KERNEL_INITIALIZER,
                                  name=prefix + 'se_expand')(se_tensor)
        x = multiply([x, se_tensor], name=prefix + 'se_excite')

    # Output phase
    x = Conv2D(block_args.output_filters, 1,
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name=prefix + 'project_conv')(x)
    x = BatchNormalization(axis=bn_axis, name=prefix + 'project_bn')(x)
    if block_args.id_skip and all(
            s == 1 for s in block_args.strides
    ) and block_args.input_filters == block_args.output_filters:
        if drop_rate and (drop_rate > 0):
            x = Dropout(drop_rate)(x)
        x = add([x, inputs], name=prefix + 'add')

    return x

def EfficientNet(width_coefficient,
                 depth_coefficient,
                 default_resolution,
                 dropout_rate=0.0,
                 drop_connect_rate=0.2,
                 depth_divisor=8,
                 blocks_args=DEFAULT_BLOCKS_ARGS,
                 model_name='efficientnet',
                 include_top=True,
                 input_tensor=None,
                 input_shape=None,
                 pooling=None,
                 classes=5,
                 **kwargs):
    """Instantiates the EfficientNet architecture using given scaling coefficients.
    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    # Arguments
        width_coefficient: float, scaling coefficient for network width.
        depth_coefficient: float, scaling coefficient for network depth.
        default_resolution: int, default input image size.
        dropout_rate: float, dropout rate before final classifier layer.
        drop_connect_rate: float, dropout rate at skip connections.
        depth_divisor: int.
        blocks_args: A list of BlockArgs to construct block modules.
        model_name: string, model name.
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor
            (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False.
            It should have exactly 3 inputs channels.
        pooling: optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
    # Returns
        A Keras model instance.
    """
    global backend, layers, models, keras_utils
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    #input = Input(shape=(img_rows, img_cols, channel,))

    img_input = Input(shape=(512, 512, 3,))

    bn_axis = 3 if K.set_image_data_format == 'channels_last' else 1
    activation = get_swish(**kwargs)

    # Build stem
    x = img_input
    x = Conv2D(round_filters(32, width_coefficient, depth_divisor), 3,
                      strides=(2, 2),
                      padding='same',
                      use_bias=False)(x)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation(activation)(x)

    # Build blocks
    num_blocks_total = sum(block_args.num_repeat for block_args in blocks_args)
    block_num = 0
    for idx, block_args in enumerate(blocks_args):
        assert block_args.num_repeat > 0
        # Update block input and output filters based on depth multiplier.
        block_args = block_args._replace(
            input_filters=round_filters(block_args.input_filters,
                                        width_coefficient, depth_divisor),
            output_filters=round_filters(block_args.output_filters,
                                         width_coefficient, depth_divisor),
            num_repeat=round_repeats(block_args.num_repeat, depth_coefficient))

        # The first block needs to take care of stride and filter size increase.
        drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
        x = mb_conv_block(x, block_args,
                          activation=activation,
                          drop_rate=drop_rate,
                          prefix='block{}a_'.format(idx + 1))
        block_num += 1
        if block_args.num_repeat > 1:
            # pylint: disable=protected-access
            block_args = block_args._replace(
                input_filters=block_args.output_filters, strides=[1, 1])
            # pylint: enable=protected-access
            for bidx in xrange(block_args.num_repeat - 1):
                drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
                block_prefix = 'block{}{}_'.format(
                    idx + 1,
                    string.ascii_lowercase[bidx + 1]
                )
                x = mb_conv_block(x, block_args,
                                  activation=activation,
                                  drop_rate=drop_rate,
                                  prefix=block_prefix)
                block_num += 1

    # Build top
    x = Conv2D(round_filters(512, width_coefficient, depth_divisor), 1,
                      padding='same',
                      use_bias=False)(x)
    x = BatchNormalization(axis=bn_axis)(x)
    x = Activation(activation)(x)
    x = GlobalAveragePooling2D()(x)
    if dropout_rate and dropout_rate > 0:
      x = Dropout(dropout_rate)(x)
    x = Dense(64, activation='sigmoid')(x)
    x = Dense(n_classes, activation='sigmoid', kernel_regularizer=regularizers.l1_l2(0.00))(x)
    inputs = img_input

    # Create model.
    model = Model(inputs, x, name=model_name)

    return model

def EfficientNetB0(include_top=True,
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=5,
                   **kwargs):
    return EfficientNet(1.0, 1.0,512, 0.0,
                        model_name='efficientnet-b0',
                        include_top=include_top,
                        input_tensor=input_tensor, input_shape=input_shape,
                        pooling=pooling, classes=classes,
                        **kwargs)

model_effinet = EfficientNetB0(include_top=True)
model_effinet.summary()

model_effinet.load_weights("Volumes/surya/aptos2019-blindness-detection/weights.effi_1st_leaf.hdf5")
print("loaded")

adam=Adam(lr=1e-05, beta_1=0.9, beta_2=0.999, amsgrad=False)

# determine Loss function and Optimizer
model_effinet.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

# Training
model_history_en_aug=  model.fit_generator(train_gen, 
                              steps_per_epoch=len(x_train)//batch_size, 
                              epochs=10, verbose=1,
                              validation_data = (x_val,y_val))

# Score trained model.
scores = model_effinet.evaluate(test_generator, verbose=1)
print('Test loss {:.4f}, accuracy {:.2f}%'.format(scores[0], scores[1] * 100))

model_effinet.save_weights("Volumes/surya/aptos2019-blindness-detection/weights.effi_1st_leaf.hdf5")
print("loaded")